In [16]:
import numpy as np
from tqdm import tqdm
from joblib import Parallel, delayed

import matplotlib.pyplot as plt
import pickle

import numba

from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
# from rsq import SVCEnsemble
from sklearn.mixture import GaussianMixture
from sklearn.random_projection import GaussianRandomProjection
from sklearn.cluster import KMeans
from sklearn.multiclass import OneVsRestClassifier

import time

import warnings
warnings.simplefilter("ignore")

def stratified_sample(y, p=0.67, replace=False):
    unique_y, counts = np.unique(y, return_counts=True)
    n_per_class = np.array([int(np.math.floor(p*c)) for c in counts])
    n_per_class = np.array([max([npc, 1]) for npc in n_per_class])
    
    inds = np.array([np.random.choice(np.where(y == unique_y[i])[0], size=npc, replace=replace) for i, npc in enumerate(n_per_class)])
    
    return np.concatenate(inds)

In [17]:
from thundersvm import SVC as tSVC

In [73]:
class ProjectionSVC(SVC):
    def __init__(self, projector='gaussian', projection_kwargs={},
                 classes=None,
                 thunder=True,
                 C=1.0, kernel='rbf', degree=3, gamma='scale', 
                 coef0=0.0, shrinking=True, probability=False,
                 tol=0.001, cache_size=200, class_weight=None, verbose=False, 
                 max_iter=-1, decision_function_shape='ovr', break_ties=False, random_state=None):
        
        if projector == 'gaussian':
            self.projector = GaussianRandomProjection
        else:
            self.projector=projector

        self.projection_kwargs=projection_kwargs
        self.classes_=classes
        
        self.kernel=kernel
        self.C=C
        
        self.thunder=thunder
                
        super().__init__(C, kernel, degree, gamma, 
                 coef0, shrinking, probability,
                 tol, cache_size, class_weight, verbose, 
                 max_iter, decision_function_shape, break_ties, random_state)
        
        
    def fit(self, X, y):
        if self.classes_ is None:
            self.classes_ = np.unique(y)

        if -1 in self.classes_:
            self.classes_ = self.classes_[1:]

        self.classes_ = np.unique(y)
        
        if self.thunder:
            self.svc = OneVsRestClassifier(tSVC(kernel=self.kernel, C=self.C))
        else:
            self.svc = SVC(kernel=self.kernel, C=self.C)
        
        if self.projector is not None:
            self.projector = self.projector(**self.projection_kwargs)
            self.projector.fit(X)
            self.svc.fit(self.projector.transform(X), y)
        else:
            self.svc.fit(X, y)
        
        
    def decision_function(self, X):
        if self.projector is not None:
            dfs = self.svc.decision_function(self.projector.transform(X))
        else:
            dfs = self.svc.decision_function(X)
            
        return dfs
        
    
    def predict(self, X):
        if self.projector is None:
            return self.classes_[np.argmax(self.decision_function(X), axis=-1)]
        else:
            return self.classes_[np.argmax(self.decision_function(self.projector.transform(X)), axis=-1)]
        
        
class SemiSupervisedSVC(ProjectionSVC):
    def __init__(self, projector=None, projection_kwargs={}, 
                 classes=None, 
                 thunder=True,
                 cluster_class='gmm', cluster_kwargs={'n_components':-2, 'covariance_type':'spherical'},
                 C=1.0, kernel='rbf', degree=3, gamma='scale', 
                 coef0=0.0, shrinking=True, probability=False,
                 tol=0.001, cache_size=200, class_weight=None, verbose=False, 
                 max_iter=-1, decision_function_shape='ovr', break_ties=False, random_state=None):
        
        if cluster_class == 'gmm':
            self.cluster_class = GaussianMixture
        elif cluster_class == 'kmeans':
            self.cluster_class = KMeans
        else:
            self.cluster_class=cluster_class
            
        self.cluster_kwargs=cluster_kwargs
        
        super().__init__(projector, projection_kwargs, classes, thunder, C, kernel, degree, gamma, 
                 coef0, shrinking, probability,
                 tol, cache_size, class_weight, verbose, 
                 max_iter, decision_function_shape, break_ties, random_state)
        
        
    def fit(self, X, y, y_induced=None):
        self.unlabeled_inds = np.where(y == -1)[0].astype(int)
        self.labeled_inds = np.where(y != -1)[0].astype(int)
        
        if self.classes_ is None:
            self.classes_ = np.unique(y)

            if -1 in self.classes_:
                self.classes_ = self.classes_[1:]

        
        n_classes = len(self.classes_)
        labeled_inds = np.where(y != 1)[0]
        
        if y_induced is None:
            if 'n_components' in list(self.cluster_kwargs.keys()):
                if self.cluster_kwargs['n_components'] < 0:
                    self.cluster_kwargs['n_components'] *= -1
                    self.cluster_kwargs['n_components'] *= len(self.classes_)
                    self.cluster_kwargs['n_components'] = int(self.cluster_kwargs['n_components'])
                self.n_components=self.cluster_kwargs['n_components']
            elif 'n_clusters' in list(self.cluster_kwargs.keys()):
                if self.cluster_kwargs['n_clusters'] < 0:
                    self.cluster_kwargs['n_clusters'] *= -1
                    self.cluster_kwargs['n_clusters'] *= len(self.classes_)
                    self.cluster_kwargs['n_clusters'] = int(self.cluster_kwargs['n_clusters'])
                self.n_components=self.cluster_kwargs['n_clusters']
            else:
                raise ValueError('keyword n_components or n_clusters must be present in cluster_kwargs')
        else:
            self.n_components = len(np.unique(y_induced))
            
                
        if y_induced is None:
            cluster_instance = self.cluster_class(**self.cluster_kwargs)
        
        if self.projector is None:
            if y_induced is None:
                y_induced = cluster_instance.fit_predict(X)
        else:
            self.projector = self.projector(**self.projection_kwargs) 
            self.projector.fit(X)
            if y_induced is None:
                y_induced = cluster_instance.fit_predict(self.projector.transform(X))
         
        self.labels_per_cluster = np.zeros((self.n_components, n_classes))
        
        for i in range(n_classes):
            inds = np.where(y == self.classes_[i])
            temp_y_induced = y_induced[inds]
            temp_counts = np.array([len(np.where(temp_y_induced == c)[0]) for c in range(self.n_components)])
            
            self.labels_per_cluster[:, i] = temp_counts
 
        self.svc = SVC(kernel=self.kernel, C=self.C)
        
        if self.projector is None:
            self.svc.fit(X, y_induced)
        else:
            self.svc.fit(self.projector.transform(X), y_induced)
    

    def decision_function(self, X):
        n_labeled = np.sum(self.labels_per_cluster)
        
        if self.projector is None:
            dfs = self.svc.decision_function(X)
        else:
            dfs = self.svc.decision_function(self.projector.transform(X))
            
        return dfs @ self.labels_per_cluster
            

class SVCEnsemble(SVC):
    def __init__(self, n_sup=1, n_semisup=1, sup_weight=None, 
                 p_inbag=1, 
                 projector=None, projection_kwargs={},
                 classes=None, n_jobs=1,
                 thunder=False, 
                 induce=0, cluster_class='gmm', cluster_kwargs={'n_components':-2, 'covariance_type':'spherical'},
                 C=1.0, kernel='rbf', degree=3, gamma='scale', 
                 coef0=0.0, shrinking=True, probability=False,
                 tol=0.001, cache_size=200, class_weight=None, verbose=False, 
                 max_iter=-1, decision_function_shape='ovr', break_ties=False, random_state=None):
        
        self.induce=induce
        self.n_sup=n_sup
        self.n_semisup = n_semisup
        self.sup_weight=sup_weight

        self.p_inbag=p_inbag
        
        self.classes_ = classes
        self.n_jobs=n_jobs
        
        if projector is "gaussian":
            self.projector=GaussianRandomProjection
        else:
            self.projector=projector

        self.projection_kwargs=projection_kwargs
        
        if cluster_class == 'gmm':
            self.cluster_class = GaussianMixture
        elif cluster_class == 'kmeans':
            self.cluster_class = KMeans
            
        self.cluster_kwargs=cluster_kwargs
        
        self.kernel=kernel
        self.C=C
        
        self.thunder=True
            
        self.ensemble = []
        
        super().__init__(C, kernel, degree, gamma, 
                 coef0, shrinking, probability,
                 tol, cache_size, class_weight, verbose, 
                 max_iter, decision_function_shape, break_ties, random_state)
        
        
    def fit(self, X, y):
        self.unlabeled_inds = np.where(y == -1)[0].astype(int)
        self.labeled_inds = np.where(y != -1)[0].astype(int)
        
        if self.sup_weight is None:
            self.sup_weight = len(self.labeled_inds) / (len(self.labeled_inds) + len(self.unlabeled_inds))
        
        if self.classes_ is None:
            self.classes_ = np.unique(y)

            if -1 in self.classes_:
                self.classes_ = self.classes_[1:]
                
        if self.induce > 0:
            if 'n_components' in list(self.cluster_kwargs.keys()):
                if self.cluster_kwargs['n_components'] < 0:
                    self.cluster_kwargs['n_components'] *= -1
                    self.cluster_kwargs['n_components'] *= len(self.classes_)
                    self.cluster_kwargs['n_components'] = int(self.cluster_kwargs['n_components'])
                self.n_components=self.cluster_kwargs['n_components']
            elif 'n_clusters' in list(self.cluster_kwargs.keys()):
                if self.cluster_kwargs['n_clusters'] < 0:
                    self.cluster_kwargs['n_clusters'] *= -1
                    self.cluster_kwargs['n_clusters'] *= len(self.classes_)
                    self.cluster_kwargs['n_clusters'] = int(self.cluster_kwargs['n_clusters'])
                self.n_components=self.cluster_kwargs['n_clusters']
            else:
                raise ValueError('keyword n_components must be present in cluster_kwargs')
            
            cluster_instance = self.cluster_class(**self.cluster_kwargs)
            self.y_induced=cluster_instance.fit_predict(X)
                        
        condensed_func = lambda x: self._train_svc(X, y, x, stratified=True)
        func_tuples = np.concatenate((np.ones(self.n_sup), np.zeros(self.n_semisup))).astype(int)
        
        self.ensemble = Parallel(n_jobs=self.n_jobs)(delayed(condensed_func)(tuple_) for tuple_ in func_tuples)
            
    def _train_svc(self, X, y, supervised=True, stratified=True):                
        if len(self.labeled_inds) == len(y):
            all_supervised=True
        else:
            all_supervised=False

        if self.p_inbag >= 1:
            self.p_inbag=1
            replace=False
        else:
            replace=True
            
        if supervised:
            bag_inds = stratified_sample(y[self.labeled_inds], p=self.p_inbag, replace=False)
            svc = ProjectionSVC(projector=self.projector, projection_kwargs=self.projection_kwargs, 
                                thunder=self.thunder, classes=self.classes_, kernel=self.kernel, C=self.C)
            svc.fit(X[self.labeled_inds[bag_inds]], y[self.labeled_inds[bag_inds]])
            
        else:
            sbag_inds = stratified_sample(y[self.labeled_inds], p=self.p_inbag, replace=False)
            if all_supervised:
                bag_inds = sbag_inds
            else:
                ssbag_inds = np.random.choice(len(self.unlabeled_inds), size=int(X.shape[0]*0.67), replace=True)
                bag_inds = np.concatenate((self.labeled_inds[sbag_inds], ssbag_inds))
            
            svc = SemiSupervisedSVC(projector=self.projector, projection_kwargs=self.projection_kwargs, 
                                    classes=self.classes_,
                                    thunder=self.thunder,
                                   cluster_class=self.cluster_class, cluster_kwargs=self.cluster_kwargs,
                                    kernel=self.kernel, C=self.C)

            use_pre_induced=np.random.binomial(1, self.induce)
            if use_pre_induced:
                svc.fit(X[bag_inds], y[bag_inds], self.y_induced[bag_inds])
            else:
                svc.fit(X[bag_inds], y[bag_inds])
                    
                
        return svc
    
    def decision_function(self, X):
        if self.n_sup > 0:
            sup_dfs = np.sum([svc.decision_function(X) for svc in self.ensemble[:self.n_sup]], axis=0)
            
            sup_dfs /= np.sqrt((sup_dfs ** 2).sum(axis=-1, keepdims=True))
        else:
            sup_dfs = 0
        

        if self.n_semisup > 0:    
            semisup_dfs = np.sum([svc.decision_function(X) for svc in self.ensemble[self.n_sup:]], axis=0)
            semisup_dfs /= np.sqrt((semisup_dfs ** 2).sum(axis=-1, keepdims=True))
        else:
            semisup_dfs = 0

        return (self.sup_weight * sup_dfs) + ((1 - self.sup_weight) * semisup_dfs)
    
    def predict_proba(self, X):
        return self.decision_function(X)
    
    
    def predict(self, X):
        return np.argmax(self.decision_function(X), axis=1)

In [74]:
def few_shot_gaussians(n, d, n_classes=2, type2_var=0.5, var=1, n_labels_per_class=1, n_test=100, acorn=None):
    if acorn is not None:
        np.random.seed(acorn)
                
    means = np.random.multivariate_normal(np.zeros(d), type2_var * np.eye(d), size=int(n_classes))
            
    data = np.concatenate([np.random.multivariate_normal(mean, var*np.eye(d), size=n) for mean in means])
    latent_labels = np.concatenate([i*np.ones(n) for i in range(n_classes)])
    
    data_test = np.concatenate([np.random.multivariate_normal(mean, var*np.eye(d), size=n_test) for mean in means])
    y_test = np.concatenate([i*np.ones(n_test) for i in range(n_classes)])
    
    shuf = np.random.choice(n_classes*n, n_classes*n, replace=False)
    data, latent_labels = data[shuf], latent_labels[shuf]
    
    known_labels = -1 * np.ones(n_classes*n)
    labeled_inds = [np.where(latent_labels == i)[0][:n_labels_per_class] for i in range(n_classes)]
    
    for i, inds in enumerate(labeled_inds):
        known_labels[inds] = i
                
    return data, known_labels, latent_labels, data_test, y_test

In [81]:
def random_forest_exp(n=1000, d=10, n_classes=10, cc=0.5, c=0.75, n_labels_per_class=50, acorn=None, algorithm_indices=np.arange(4)):
    accs = np.zeros(len(algorithm_indices))
    
    X, y, y_true, X_test, y_test = few_shot_gaussians(n, d, n_classes, cc, c, int(n*p), 100, acorn)
    
    inds_sup = np.where(y != -1)[0].astype(int)
    inds_unsup = np.array([i for i in range(len(y)) if i not in inds_sup])

    y_ = -1 * np.ones(int(n*n_classes))
    
    y_[inds_sup] = y[inds_sup]
    
    classes = np.arange(n_classes)
    
    #-
    accs = np.zeros(6)
    times = np.zeros(6)
            
    time_ = time.time()
    
                                                                            
    #- GMM, 0.5*n_classes
    svc_semisup_ensemble6 = SVCEnsemble(n_sup=200, n_semisup=200, p_inbag=0.67,
                                        projection_kwargs={'n_components': 8}, 
                                        induce=0,
                                        cluster_class='gmm', cluster_kwargs={'n_components': -2},
                                        kernel='linear',
                                        thunder=True,
                                        classes=classes)
    svc_semisup_ensemble6.fit(X, y)
    accs[0] = (svc_semisup_ensemble6.predict(X_test) == y_test).mean()
    
    times[0] = time.time() - time_
    time_ = time.time()
    
    #- GMM, 0.5*n_classes
    svc_semisup_ensemble6 = SVCEnsemble(n_sup=200, n_semisup=200, p_inbag=0.67,
                                        projection_kwargs={'n_components': 8},
                                        induce=0.25, 
                                        cluster_class='gmm', cluster_kwargs={'n_components': -2},
                                        kernel='linear',
                                        thunder=True,
                                        classes=classes)
    svc_semisup_ensemble6.fit(X, y)
    accs[1] = (svc_semisup_ensemble6.predict(X_test) == y_test).mean()
    
    times[1] = time.time() - time_
    time_ = time.time()
    
    #- GMM, 0.5*n_classes
    svc_semisup_ensemble6 = SVCEnsemble(n_sup=200, n_semisup=200, p_inbag=0.67,
                                        projection_kwargs={'n_components': 8},
                                        induce=0, 
                                        cluster_class='gmm', cluster_kwargs={'n_components': -2},
                                        kernel='rbf',
                                        thunder=False,
                                        classes=classes)
    svc_semisup_ensemble6.fit(X, y)
    accs[2] = (svc_semisup_ensemble6.predict(X_test) == y_test).mean()
    
    times[2] = time.time() - time_
    time_ = time.time()
                                                                             
    
#     try:
#         #- LDA
#         lda = LDA()
#         lda.fit(X[inds_sup], y[inds_sup])
#         accs[-1] = (lda.predict(X_test) == y_test).mean()
#     except:
#         accs[-1] = 1 / n_classes
    
    return accs, times

In [82]:
run_experiment=True

np.random.seed(2)

n_mc = 90
n_cores=90

props = [0.01, 0.025, 0.05, 0.1, 0.25, 0.5, 1]
n=100
d=20
c=0.5
cc = 0.25
n_classes=10

# algorithm_indices=np.arange(5)
accs = np.zeros((len(props), 6, n_mc))
times = np.zeros((len(props), 6, n_mc))

if run_experiment:
    for i, p in enumerate(tqdm(props)):
        seeds = np.random.randint(0, 10000, size=n_mc)
        condensed_func = lambda x: random_forest_exp(n, d, n_classes, cc, c, int(n*p), x)

        accs_and_times = Parallel(n_jobs=90)(delayed(condensed_func)(x) for x in seeds)
        accs[i] = np.array([accs_and_times[i][0] for i in range(n_mc)]).T
        times[i] = np.array([accs_and_times[i][1] for i in range(n_mc)]).T
        
        print(np.mean(accs[i], axis=-1))
        print(np.mean(times[i], axis=-1))

 14%|█▍        | 1/7 [02:35<15:33, 155.61s/it]

[0.72785556 0.72574444 0.7151     0.         0.         0.        ]
[44.48121653 41.32407547 55.18655025  0.          0.          0.        ]


 14%|█▍        | 1/7 [04:02<24:14, 242.41s/it]


KeyboardInterrupt: 

In [ ]:
fig, ax = plt.subplots(1,1)

labels=['Sup', 'Semisup', 
        'Comb. (0.5)', 'Comb. (Ratio)', 
        'Comb. + Ens. (0.5)', 'Comb. + Ens. (Ratio)',
        'Comb. + Ens. (0.5) + RP', 'Comb. + Ens. (Ratio) + RP',
       ' LDA'
       ]

run_experiment=True

if run_experiment:
    means = np.mean(accs, axis=-1)
    stds = np.std(accs, axis=-1) / np.sqrt(n_mc)

#     pickle.dump(means, open('data/forests_meanaccs_fewshotgaussians.pkl', 'wb'))
#     pickle.dump(stds, open('data/forests_stdaccs_fewshotgaussians.pkl', 'wb'))
    
else:
#     means = pickle.load(open('data/forests_meanaccs_fewshotgaussians.pkl', 'rb'))
#     stds = pickle.load(open('data/forests_stdaccs_fewshotgaussians.pkl', 'rb'))
    pass

for i, mean in enumerate(means.T):
    ax.scatter(props, mean, label=labels[i])
    ax.errorbar(props, mean, yerr=stds.T[i])

ax.set_xscale('log')
ax.legend()
ax.set_xlabel('Proportion of labeled data')
ax.set_ylabel('Accuracy')

In [ ]:
class SemiSupervisedSVC2(SVC):
    def __init__(self, sC=1.0, 
                 kernel='rbf', degree=3, gamma='scale', 
                 coef0=0.0, shrinking=True, probability=False,
                 tol=0.001, cache_size=200, class_weight=None, verbose=False, 
                 max_iter=-1, decision_function_shape='ovr', break_ties=False, random_state=None):
        
        self.sup_weight=sup_weight
        self.sup_cv_folds=sup_cv_folds
        
        super().__init__(C, kernel, degree, gamma, 
                 coef0, shrinking, probability,
                 tol, cache_size, class_weight, verbose, 
                 max_iter, decision_function_shape, break_ties, random_state)
        
        
    def fit(self, X, y):
        self.unlabeled_inds = np.where(y == -1)[0].astype(int)
        self.labeled_inds = np.where(y != -1)[0].astype(int)
        
        self.classes_ = np.unique(y)
        
        if -1 in self.classes_:
            self.classes_ = self.classes_[1:]
                
        self.sup_svc = self._fit_supervised(X[self.labeled_inds], y[self.labeled_inds])
        self.semisup_svc = self._fit_semisupervised(X, y)
        
    
    def _fit_supervised(self, X, y):
        svc = SVC()
        svc.fit(X,y)
        return svc
        
        
    def _fit_semisupervised(self, X, y):
        n_classes = len(self.classes_)
        labeled_inds = np.where(y != 1)[0]
        
        _, counts = np.unique(y[labeled_inds], return_counts=True)
        
        gmm = GMM(min_components=2*n_classes, max_components=2*n_classes)
        induced_labels = gmm.fit_predict(X)
        
        self.labels_per_cluster = np.zeros((2*n_classes, n_classes))
        
        for i in range(n_classes):
            inds = np.where(y == self.classes_[i])
            temp_induced_labels = induced_labels[inds]
            temp_counts = np.array([len(np.where(temp_induced_labels == c)[0]) for c in range(2*n_classes)])
            
            self.labels_per_cluster[:, i] = temp_counts
                    
                    
        svc = SVC()
        svc.fit(X, induced_labels)

        return svc
    

    def _semisup_decision_function(self, X):
        n_induced_classes, n_classes = self.labels_per_cluster.shape
        n_labeled = np.sum(self.labels_per_cluster)
        dfs = self.semisup_svc.decision_function(X)
                
        return dfs @ self.labels_per_cluster
            
    
    def predict(self, X):
        return np.argmax(self.decision_function(X), axis=1)
        
    
    def decision_function(self, X):
        sup_dfs = self.sup_svc.decision_function(X)
        sup_dfs /=  np.sqrt((sup_dfs ** 2).sum(axis=-1, keepdims=True))
        
        semisup_dfs = self._semisup_decision_function(X)
        semisup_dfs /=  np.sqrt((semisup_dfs ** 2).sum(axis=-1, keepdims=True))
                
        return (self.sup_weight * sup_dfs) + (1 - self.sup_weight) * semisup_dfs